# 🚀 Google Colab Setup

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ogautier1980/sandbox-ml/blob/main/cours/03_regression/03_demo_regression_lineaire.ipynb)

**Si vous exécutez ce notebook sur Google Colab**, exécutez la cellule suivante pour installer les dépendances.

In [ ]:
# Installation des dépendances (Google Colab uniquement)import sysIN_COLAB = 'google.colab' in sys.modulesif IN_COLAB:    print('📦 Installation des packages...')        # Packages ML de base    !pip install -q numpy pandas matplotlib seaborn scikit-learn        # Détection du chapitre et installation des dépendances spécifiques    notebook_name = '03_demo_regression_lineaire.ipynb'  # Sera remplacé automatiquement        # Ch 06-08 : Deep Learning    if any(x in notebook_name for x in ['06_', '07_', '08_']):        !pip install -q torch torchvision torchaudio        # Ch 08 : NLP    if '08_' in notebook_name:        !pip install -q transformers datasets tokenizers        if 'rag' in notebook_name:            !pip install -q sentence-transformers faiss-cpu rank-bm25        # Ch 09 : Reinforcement Learning    if '09_' in notebook_name:        !pip install -q gymnasium[classic-control]        # Ch 04 : Boosting    if '04_' in notebook_name and 'boosting' in notebook_name:        !pip install -q xgboost lightgbm catboost        # Ch 05 : Clustering avancé    if '05_' in notebook_name:        !pip install -q umap-learn        # Ch 11 : Séries temporelles    if '11_' in notebook_name:        !pip install -q statsmodels prophet        # Ch 12 : Vision avancée    if '12_' in notebook_name:        !pip install -q ultralytics timm segmentation-models-pytorch        # Ch 13 : Recommandation    if '13_' in notebook_name:        !pip install -q scikit-surprise implicit        # Ch 14 : MLOps    if '14_' in notebook_name:        !pip install -q mlflow fastapi pydantic        print('✅ Installation terminée !')else:    print('ℹ️  Environnement local détecté, les packages sont déjà installés.')

# Chapitre 03 - Régression Linéaire

**Objectifs :**
- Comprendre et implémenter la régression linéaire simple
- Maîtriser la régression linéaire multiple
- Utiliser la régression polynomiale
- Diagnostiquer les résidus
- Évaluer les performances (R², MSE, RMSE)

**Prérequis :** Chapitres 00, 01, 02

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.data  # type: ignoresets import load_diabetes, make_regression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.pipeline import Pipeline
import scipy.stats as stats

# Configuration
np.random.seed(42)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Imports réussis")

## 1. Régression Linéaire Simple

### 1.1 Génération de données synthétiques

Créons des données avec une relation linéaire : $y = 3x + 5 + \epsilon$

In [ ]:
# Générer des données synthétiques
n_samples = 100
X_simple = np.linspace(0, 10, n_samples).reshape(-1, 1)
y_true = 3 * X_simple.ravel() + 5
noise = np.random.normal(0, 2, n_samples)
y_simple = y_true + noise

# Visualisation
plt.figure(figsize=(10, 6))
plt.scatter(X_simple, y_simple, alpha=0.6, label='Données observées')
plt.plot(X_simple, y_true, 'r--', label='Vraie relation (y=3x+5)', linewidth=2)
plt.xlabel('X')
plt.ylabel('y')
plt.title('Régression Linéaire Simple - Données Synthétiques')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Nombre d'échantillons : {n_samples}")
print(f"Range de X : [{X_simple.min():.2f}, {X_simple.max():.2f}]")
print(f"Range de y : [{y_simple.min():.2f}, {y_simple.max():.2f}]")

### 1.2 Entraînement du modèle avec scikit-learn

In [ ]:
# Créer et entraîner le modèle
model_simple = LinearRegression()
model_simple.fit(X_simple, y_simple)

# Prédictions
y_pred = model_simple.predict(X_simple)

# Paramètres appris
w1 = model_simple.coef_[0]
w0 = model_simple.intercept_

print("=== Paramètres Appris ===")
print(f"Pente (w1) : {w1:.4f} (vraie valeur: 3.0)")
print(f"Intercept (w0) : {w0:.4f} (vraie valeur: 5.0)")
print(f"\nÉquation : ŷ = {w1:.4f}x + {w0:.4f}")

### 1.3 Évaluation du modèle

In [ ]:
# Métriques
mse = mean_squared_error(y_simple, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_simple, y_pred)
r2 = r2_score(y_simple, y_pred)

print("=== Métriques de Performance ===")
print(f"MSE  : {mse:.4f}")
print(f"RMSE : {rmse:.4f}")
print(f"MAE  : {mae:.4f}")
print(f"R²   : {r2:.4f}")
print(f"\nInterprétation : Le modèle explique {r2*100:.2f}% de la variance")

### 1.4 Visualisation des prédictions

In [ ]:
plt.figure(figsize=(12, 5))

# Subplot 1 : Droite de régression
plt.subplot(1, 2, 1)
plt.scatter(X_simple, y_simple, alpha=0.6, label='Données')
plt.plot(X_simple, y_pred, 'g-', label=f'Régression (R²={r2:.3f})', linewidth=2)
plt.plot(X_simple, y_true, 'r--', label='Vraie relation', linewidth=2, alpha=0.5)
plt.xlabel('X')
plt.ylabel('y')
plt.title('Droite de Régression')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 2 : Résidus
residus = y_simple - y_pred
plt.subplot(1, 2, 2)
plt.scatter(y_pred, residus, alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--', linewidth=2)
plt.xlabel('Valeurs Prédites')
plt.ylabel('Résidus (y - ŷ)')
plt.title('Analyse des Résidus')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Moyenne des résidus : {residus.mean():.6f} (devrait être ~0)")
print(f"Écart-type des résidus : {residus.std():.4f}")

### 1.5 Implémentation manuelle (pour comprendre les maths)

Calculons les paramètres avec les formules analytiques :

$$w_1 = \frac{\text{Cov}(x, y)}{\text{Var}(x)}, \quad w_0 = \bar{y} - w_1 \bar{x}$$

In [ ]:
# Implémentation manuelle
x_flat = X_simple.ravel()
x_mean = x_flat.mean()
y_mean = y_simple.mean()

# Formule analytique
covariance = np.sum((x_flat - x_mean) * (y_simple - y_mean))
variance_x = np.sum((x_flat - x_mean)**2)

w1_manual = covariance / variance_x
w0_manual = y_mean - w1_manual * x_mean

print("=== Comparaison Implémentations ===")
print(f"scikit-learn : w1={w1:.6f}, w0={w0:.6f}")
print(f"Manuel       : w1={w1_manual:.6f}, w0={w0_manual:.6f}")
print(f"\nDifférence   : w1={abs(w1-w1_manual):.10f}, w0={abs(w0-w0_manual):.10f}")

## 2. Régression Linéaire Multiple

### 2.1 Dataset Diabetes (scikit-learn)

Prédisons la progression du diabète à partir de 10 features biomédicales.

In [ ]:
# Charger le dataset
diabetes = load_diabetes()
X = diabetes.data  # type: ignore
y = diabetes.target  # type: ignore
feature_names = diabetes.feature_names  # type: ignore

print("=== Dataset Diabetes ===")
print(f"Nombre d'échantillons : {X.shape[0]}")
print(f"Nombre de features : {X.shape[1]}")
print(f"\nFeatures : {feature_names}")
print(f"\nStatistiques de la cible (y) :")
print(f"  Min : {y.min():.2f}")
print(f"  Max : {y.max():.2f}")
print(f"  Moyenne : {y.mean():.2f}")
print(f"  Écart-type : {y.std():.2f}")

# Visualiser les corrélations
df = pd.DataFrame(X, columns=feature_names)
df['target'] = y

plt.figure(figsize=(12, 10))
sns.heatmap(df.corr(), annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Matrice de Corrélation - Dataset Diabetes')
plt.tight_layout()
plt.show()

### 2.2 Split Train/Test et Normalisation

In [ ]:
# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("=== Split Train/Test ===")
print(f"Train : {X_train.shape[0]} échantillons")
print(f"Test  : {X_test.shape[0]} échantillons")

# Normalisation (optionnelle mais recommandée)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n✅ Données normalisées (moyenne=0, std=1)")

### 2.3 Entraînement du modèle

In [ ]:
# Modèle de régression linéaire multiple
model_multi = LinearRegression()
model_multi.fit(X_train_scaled, y_train)

# Prédictions
y_train_pred = model_multi.predict(X_train_scaled)
y_test_pred = model_multi.predict(X_test_scaled)

# Métriques
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

print("=== Performance du Modèle ===")
print(f"Train R² : {train_r2:.4f}")
print(f"Test R²  : {test_r2:.4f}")
print(f"\nTrain RMSE : {train_rmse:.2f}")
print(f"Test RMSE  : {test_rmse:.2f}")

if abs(train_r2 - test_r2) < 0.1:
    print("\n✅ Pas de surapprentissage détecté")
else:
    print("\n⚠️  Possible surapprentissage")

### 2.4 Importance des Features (Coefficients)

In [ ]:
# Coefficients du modèle
coefficients = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': model_multi.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print("=== Coefficients du Modèle ===")
print(coefficients.to_string(index=False))
print(f"\nIntercept : {model_multi.intercept_:.2f}")

# Visualisation
plt.figure(figsize=(10, 6))
plt.barh(coefficients['Feature'], coefficients['Coefficient'])
plt.xlabel('Coefficient')
plt.title('Importance des Features (Coefficients de Régression)')
plt.axvline(x=0, color='k', linestyle='--', linewidth=0.8)
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print("\nInterprétation : Les features avec les plus grands coefficients (en valeur absolue)")
print("ont le plus d'impact sur la prédiction.")

### 2.5 Visualisations Diagnostiques

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Prédictions vs Valeurs Réelles
axes[0, 0].scatter(y_test, y_test_pred, alpha=0.6)
axes[0, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
                'r--', linewidth=2, label='Prédiction parfaite')
axes[0, 0].set_xlabel('Valeurs Réelles')
axes[0, 0].set_ylabel('Valeurs Prédites')
axes[0, 0].set_title(f'Prédictions vs Réelles (Test R²={test_r2:.3f})')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Distribution des résidus
residuals_test = y_test - y_test_pred
axes[0, 1].hist(residuals_test, bins=30, edgecolor='black', alpha=0.7)
axes[0, 1].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[0, 1].set_xlabel('Résidus')
axes[0, 1].set_ylabel('Fréquence')
axes[0, 1].set_title(f'Distribution des Résidus (Test, μ={residuals_test.mean():.2f})')
axes[0, 1].grid(True, alpha=0.3, axis='y')

# 3. Résidus vs Prédictions
axes[1, 0].scatter(y_test_pred, residuals_test, alpha=0.6)
axes[1, 0].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[1, 0].set_xlabel('Valeurs Prédites')
axes[1, 0].set_ylabel('Résidus')
axes[1, 0].set_title('Résidus vs Prédictions')
axes[1, 0].grid(True, alpha=0.3)

# 4. Q-Q Plot (Normalité des résidus)
stats.probplot(residuals_test, dist="norm", plot=axes[1, 1])
axes[1, 1].set_title('Q-Q Plot (Test de Normalité des Résidus)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Analyse des diagnostics :")
print("1. Les points doivent être proches de la diagonale (plot 1)")
print("2. Les résidus doivent être centrés sur 0 (plot 2)")
print("3. Pas de pattern dans les résidus (plot 3)")
print("4. Les points doivent suivre la ligne rouge (Q-Q plot, normalité)")

## 3. Régression Polynomiale

### 3.1 Données Non-Linéaires

In [ ]:
# Générer des données avec relation polynomiale
np.random.seed(42)
X_poly = np.linspace(-3, 3, 100).reshape(-1, 1)
y_poly_true = 0.5 * X_poly.ravel()**2 - 2*X_poly.ravel() + 1
y_poly = y_poly_true + np.random.normal(0, 1, 100)

plt.figure(figsize=(10, 6))
plt.scatter(X_poly, y_poly, alpha=0.6, label='Données')
plt.plot(X_poly, y_poly_true, 'r--', label='Vraie relation (polynomiale)', linewidth=2)
plt.xlabel('X')
plt.ylabel('y')
plt.title('Données avec Relation Polynomiale')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### 3.2 Comparaison de différents degrés polynomiaux

In [ ]:
degrees = [1, 2, 5, 10]
colors = ['blue', 'green', 'orange', 'red']

plt.figure(figsize=(14, 4))

results = []

for i, degree in enumerate(degrees):
    # Créer features polynomiales
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_poly_features = poly.fit_transform(X_poly)
    
    # Entraîner modèle
    model = LinearRegression()
    model.fit(X_poly_features, y_poly)
    y_pred_poly = model.predict(X_poly_features)
    
    # Métriques
    r2 = r2_score(y_poly, y_pred_poly)
    rmse = np.sqrt(mean_squared_error(y_poly, y_pred_poly))
    results.append({'Degree': degree, 'R²': r2, 'RMSE': rmse})
    
    # Subplot
    plt.subplot(1, 4, i+1)
    plt.scatter(X_poly, y_poly, alpha=0.4, s=20)
    plt.plot(X_poly, y_pred_poly, color=colors[i], linewidth=2, 
             label=f'Degré {degree}')
    plt.plot(X_poly, y_poly_true, 'k--', linewidth=1, alpha=0.5, label='Vraie')
    plt.title(f'Degré {degree}\nR²={r2:.3f}')
    plt.xlabel('X')
    if i == 0:
        plt.ylabel('y')
    plt.legend(fontsize=8)
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Tableau comparatif
print("\n=== Comparaison des Modèles ===")
df_results = pd.DataFrame(results)
print(df_results.to_string(index=False))

print("\nObservation : Degré 2 capture bien la relation. Degré > 5 surapprentissage.")

### 3.3 Pipeline complet avec Validation Croisée

In [ ]:
# Tester différents degrés avec validation croisée
degrees_cv = range(1, 11)
cv_scores = []

for degree in degrees_cv:
    # Pipeline : Polynomiale + Scaling + Régression
    pipeline = Pipeline([
        ('poly', PolynomialFeatures(degree=degree, include_bias=False)),
        ('scaler', StandardScaler()),
        ('regressor', LinearRegression())
    ])
    
    # Validation croisée 5-fold
    scores = cross_val_score(pipeline, X_poly, y_poly, cv=5, 
                             scoring='r2')
    cv_scores.append(scores.mean())

# Visualisation
plt.figure(figsize=(10, 6))
plt.plot(degrees_cv, cv_scores, 'o-', linewidth=2, markersize=8)
best_degree = degrees_cv[np.argmax(cv_scores)]
plt.axvline(x=best_degree, color='r', linestyle='--', linewidth=2, 
            label=f'Meilleur degré: {best_degree}')
plt.xlabel('Degré du Polynôme')
plt.ylabel('R² (Validation Croisée)')
plt.title('Sélection du Degré Polynomial par Validation Croisée')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(degrees_cv)
plt.show()

print(f"\n✅ Meilleur degré : {best_degree}")
print(f"R² moyen : {max(cv_scores):.4f}")

## 4. Récapitulatif et Points Clés

### Points à retenir :

1. **Régression linéaire simple** : Une feature, solution analytique directe
2. **Régression linéaire multiple** : Plusieurs features, matrice de design
3. **Régression polynomiale** : Transforme les features pour capturer les non-linéarités
4. **Métriques** : R², MSE, RMSE, MAE
5. **Diagnostics** : Analyse des résidus cruciale (normalité, homoscédasticité)
6. **Validation croisée** : Essentielle pour détecter le surapprentissage

### Prochaine étape :

Voir **03_demo_regularisation.ipynb** pour Ridge, Lasso, Elastic Net

In [ ]:
print("✅ Notebook terminé !")
print("\nVous maîtrisez maintenant :")
print("  - Régression linéaire simple et multiple")
print("  - Régression polynomiale")
print("  - Évaluation et diagnostic de modèles")
print("  - Validation croisée")